# Google Search Console - Time Series

> Für diese Notebooks benötigt ihr die Sqlite mit den GSC Daten.

In [ ]:
%run helpers/code_toggle.py

In [ ]:
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import dataset
register_matplotlib_converters()

## Die Daten aus der Datenbank laden
Zunächst laden wir die Daten der gewünschten Tabelle aus unserer Datenbank. In diesem Fall wollen wir gerne die Gesamtwerte laden (ohne Suchanfragen oder Seiten). Hier ist die Datenmenge nicht sehr groß, daher laden wir mit `pd.read_sql_table` einfach die komplette Tabelle.

In [ ]:
db = dataset.connect('sqlite:///data/serienjunkies.db')
db.tables

In [ ]:
totals = pd.read_sql_table('web_country_device', con=db.engine, parse_dates=['date'])
totals.head()

## Daten gruppieren
In unserem Beispiel wollen wir die Daten auf Geräte gruppieren. Dies ist mittels `groupby` [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) ganz einfach möglich.

In [ ]:
totals.groupby('device')['clicks','impressions'].sum()

### CTR und gewichtete Position
Die CTR und Position die wir aus der Datenbank laden, können wir nicht einfach summieren. Die CTR muss aus den aggregierten Klicks & Impressionen neu errechnet werden. Ebenso muss die Position mit den Impressionen gewichtet werden, um korrekte Werte zu erhalten.

#### Hilfsfunktionen mittels Cell Magic
Wir laden daher mittels `cell magic` ein paar Funktionen aus einer Python Datei im Ordner `helpers`.

In [ ]:
%load -r 4:28 helpers/helpers.py

#### Berechnen der Spalten
Nun da wir die Funktionen geladen und die Zelle ausgeführt haben, können sie mittels `pipe`-Funktion [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pipe.html) verkettet ausgeführt werden *(method-chaining)*.

Die nächste Zelle führt also folgende Funktionen aus:
1. Neue Spalte hinzufügen. Produkt aus Position und Impression.
2. Gruppierung auf Gerät und summieren aller Metriken.
3. Berechnen der gewichteten Position.
4. Berechnen der CTR aus summierten Werten.
5. Entfernen der Spalte `pos_imp`

In [ ]:
totals.pipe(assign_pos_imp) \
    .groupby('device')['clicks','impressions', 'pos_imp'].sum() \
    .pipe(assign_position) \
    .pipe(assign_ctr) \
    .drop('pos_imp', axis='columns')

## Klicks nach Geräten im Zeitverlauf
Bisher haben wir "nur" die Gesamtwerte für die Geräte über den gesamten Zeitraum berechnet. Nachfolgend wollen wir die Daten der Geräte im Zeitverlauf darstellen. Dafür müssen wir neben `device` auch auf `date` gruppieren. Hierfür muss eine Liste, statt einem einzelnen Wert in die `groupby`-Funktion übergeben werden. 

Damit die Plots etwas hübscher aussehen, laden wir hier abermals mittels `cell magic` ein wenig Code.

In [ ]:
%load -r 33:38 helpers/helpers.py

In [ ]:
ax = totals.pipe(assign_pos_imp) \
    .groupby('date', as_index=False)['clicks','impressions', 'pos_imp'].sum() \
    .pipe(assign_position) \
    .pipe(assign_ctr) \
    .drop('pos_imp', axis=1) \
    .pipe((sns.lineplot, 'data'),
          x='date', y='clicks')
ax.set_title('Klicks im Zeitverlauf', x=0, ha='left')
sns.despine()

### Gruppierung mit Datum und Gerät
Damit wir die Daten nicht immer wieder gruppieren, speichern wir sie nun in eine neue Variable `device_totals`. Die Gruppierung und Berechnung der Werte erfolgt wie im Beipsiel oben, dieses Mal aber mit `date` in den Gruppen. 

In [ ]:
device_totals = totals.pipe(assign_pos_imp) \
    .groupby(['date','device'], as_index=False)['clicks','impressions', 'pos_imp'].sum() \
    .pipe(assign_position) \
    .pipe(assign_ctr) \
    .drop('pos_imp', axis=1)
device_totals.head()

### Plot des Zeitverlaufs
Zur Visualisierung nutzen wir direkt das Package `seaborn` [(link)](https://seaborn.pydata.org/index.html). Der Plot ist aber ebenso direkt aus Pandas möglich.

*Achtung:* Ich nutze `seaborn` hier ebenfalls innerhalb einer `pipe`. Oftmals ist es nützlich, den Plot direkt nach einigen Aggregationsschritten verkettet auszuführen. `sns.lineplot` [(link)](https://seaborn.pydata.org/generated/seaborn.lineplot.html#seaborn.lineplot) kann natürlich auch direkt verwendet werden:
```
sns.lineplot(x='date', y='clicks', data=device_totals, hue='device')
```

In [ ]:
ax = device_totals.pipe((sns.lineplot, 'data'),
          x='date', y='clicks', hue='device')
ax.set_title('Klicks pro Gerät', x=0, ha='left')
sns.despine()

## Resampling der Daten (Monate)
Bisher haben wir zappelige Tageswerte. Pandas macht es sehr einfach, die Daten auf andere Zeiteinheiten zu aggregieren. Dies kann entweder mittels `.resample` [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html) durchgeführt werden, oder mittels `pd.Grouper` [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html) direkt als Teil eines Groupby. Für beide Funktionen muss der Index des DataFrames ein Zeit-Index sein.

In [ ]:
ax = device_totals.set_index('date') \
    .resample('M')['clicks'].sum() \
    .reset_index() \
    .pipe((sns.lineplot, 'data'),
          x='date', y='clicks')
ax.set_title('Klicks pro Monat', x=0, ha='left')
sns.despine()

In [ ]:
ax = device_totals.groupby([pd.Grouper(key='date', freq='M'),'device'])['clicks'].sum() \
    .reset_index() \
    .pipe((sns.lineplot, 'data'),
          x='date', y='clicks', hue='device')
ax.set_title('Klicks pro Monat', x=0, ha='left')
sns.despine()

## Rollender Durchschnitt
Neben der Aggregation, können wir die Daten natürlich auch im rollenden Durchschnitt darstellen. Hierfür gibt es ebenfalls eine passende Funktion `.rolling` [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html). Da wir die Werte für jede Gruppe benötigen, führen wir direkt einen Spezialfall aus.
- Zeitindex setzen
- Gruppieren auf Geräte
- Berechnen des rollenden Durchschnitts der Klicks

In [ ]:
ax = device_totals.set_index('date').groupby('device')['clicks'].rolling(28).mean() \
    .reset_index() \
    .pipe((sns.lineplot, 'data'),
          x='date', y='clicks', hue='device')
device_totals.pipe((sns.lineplot, 'data'), ax=ax, alpha=.4,
                   x='date', y='clicks', hue='device', style='device', dashes=[(2,2) for n in range(3)])
ax.set_title('Klicks im rollenden Durchschnitt', ha='left', x=0)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5)) # in diesem Beispiel ist die Legende Rechts
sns.despine()

## Prozentuale Veränderung
Zur Visualisierung der prozentualen Veränderung aggregieren wir die Daten zunächst auf Monate. Dazu im `groupby` die Frequenz 'M' für Monat nutzen. Danach die übliche Berechnung von CTR und Position.

In [ ]:
totals_agg_m = totals.pipe(assign_pos_imp) \
    .groupby(pd.Grouper(key='date', freq='M'))[['clicks','impressions','pos_imp']].sum() \
    .pipe(assign_position) \
    .pipe(assign_ctr) \
    .drop('pos_imp', axis='columns')
totals_agg_m.head()

Nun kann ganz einfach mittels `pct_change` [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pct_change.html) die prozentuale Veränderung berechnet werden.

In [ ]:
metric = 'clicks'
pct_change = totals_agg_m.pct_change() \
    .dropna() \
    .reset_index() \
    .assign(month_name = lambda x: x['date'].dt.month_name('DEU_DEU'))
pct_change['positive'] = pct_change[metric] > 0
pct_change.head()

In [ ]:
ax = pct_change.plot(kind='bar', y=metric, x='month_name', color=pct_change.positive.map({True:'g',False:'r'}))
ax.set_title(f'Prozentuale Veränderung ({metric} - 1 Monat)', x=0, ha='left')
sns.despine()

## Entwicklung Tage im Monat / Woche
Häufig bringt es interessante Erkenntnisse über das Datenset, wenn man den Zeitverlauf des gesamten Datensets auf die Tage des Monats, oder aber die Tage der Woche aggregiert. So kann man leicht erkennen, ob es in den Monaten / Wochen einen Trend gibt.

Wir aggregieren die Daten ähnlich wie bisher, fügen aber eine weitere Spalte hinzu. Einmal den Tag des Monats `pd.DataFrame['date'].dt.day` [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.day.html) und weiter unten den Tag der Woche `pd.DataFrame['date'].dt.dayofweek` [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.dayofweek.html).

In [ ]:
totals_agg_d = totals.pipe(assign_pos_imp) \
    .assign(day_in_month = lambda x: x.date.dt.day) \
    .groupby('day_in_month', as_index=False)[['clicks','impressions','pos_imp']].mean() \
    .pipe(assign_position) \
    .pipe(assign_ctr) \
    .drop('pos_imp', axis='columns')
totals_agg_d.head()

In [ ]:
ax = totals_agg_d.query('day_in_month < 31') \
    .pipe((sns.regplot, 'data'), x='day_in_month', y='clicks')
ax.set_title('Verlauf Klicks über Tage im Monat', x=0, ha='left')
sns.despine()

### Geräte und Metriken in einem facettierten Plot
Damit man sich nicht nur Klicks, oder jede Metrik einzeln anschauen muss, gibt es die Möglichkeit facettierte Plots zu erstellen. Dabei können Zeilen und Spalten innerhalb des Plots definiert werden. 

**Achtung:** Damit dies mit `seaborn` funtkioniert, müssen die Daten in ein langes (tidy) Format gebracht werden. Im Beispiel machen wir dies mit der Pandas Funktion `melt` [(link)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.day.html)

In [ ]:
sns.set('notebook', rc={'figure.constrained_layout.use':False})
g = totals.assign(day_in_month = lambda x: x.date.dt.day) \
    .groupby(['device','day_in_month'], as_index=False)[['clicks','impressions']].mean() \
    .pipe(assign_ctr) \
    .query('day_in_month < 31') \
    .melt(id_vars=['device','day_in_month'], value_vars=['clicks','impressions','ctr']) \
    .pipe((sns.FacetGrid, 'data'),
          col='device', row='variable',
          hue='device', sharey=False, height=5) \
    .map(sns.regplot, 'day_in_month', 'value', truncate=True)
g.fig.suptitle('Verlauf über Tage im Monat', fontsize=20)
plt.subplots_adjust(top=.92)
sns.despine()
g.fig.savefig('sj_days_of_month.png', bbox_inches='tight')

In [ ]:
g = totals.assign(day_in_week = lambda x: x.date.dt.dayofweek+1) \
    .groupby(['device','day_in_week'], as_index=False)[['clicks','impressions']].mean() \
    .pipe(assign_ctr) \
    .melt(id_vars=['device','day_in_week'], value_vars=['clicks','impressions','ctr']) \
    .pipe((sns.FacetGrid, 'data'),
          col='device', row='variable',
          hue='device', sharey=False, height=5) \
    .map(sns.regplot, 'day_in_week', 'value', truncate=True)
g.fig.suptitle('Verlauf über Tage der Woche', fontsize=20)
plt.subplots_adjust(top=.92)
sns.despine()
g.fig.savefig('sj_days_of_week.png', bbox_inches='tight')